In [ ]:
import pandas as pd
import ast
import re
import os
import psycopg2 as pg

In [ ]:
def clean_param(param):
    after_clean = re.sub(r"\d,22\sserialization::archive\s9\s\d\s", "", param)
    after_clean = re.sub(r"\\", "/", after_clean)
    if re.search(r"\d{1,2}\s", after_clean) == None:
        return after_clean
    else:
        return re.sub(r"\d{1,2}\s", "", after_clean)


def param_to_list(param):
    return ast.literal_eval("[" + param + "]")


def to_hex(df):
    if df.hexLocation != -1:
        hex_index = int(df.hexLocation)
        df.Parameters[hex_index] = "0x" + str(
            hex(int(df.Parameters[hex_index]))[2:].zfill(2)
        )
    return df


def file_filter(filedir, keyword):
    allfilelist = os.listdir(filedir)
    targetfiles = [file for file in allfilelist if keyword in file]
    return targetfiles


def strlist(slist):
    return "||".join(slist)

In [ ]:
code_type = {"69": "Error", "73": "Info", "87": "Warning"}
col_type = {"Type": str, "DateTime": int, "ResourceId": int, "Parameters":str}
work_path = r"D:\DataWork\AllWerfenChinaTop\BIOFLASHlog\Data"
selected_columns = ["Type", "DateTime", "ResourceId", "Parameters"]

In [ ]:
df_logs = pd.DataFrame({"filename" : file_filter((work_path + "\\InstrumentLog"), ".txt")})

In [ ]:
df_logs["sn"] = df_logs["filename"].str.split("_").str[1]
df_logs["date"] = df_logs["filename"].str.split("_").str[5]

In [ ]:
df_logs.sort_values(["sn", "date"], inplace=True)

In [ ]:
dferror = pd.read_excel((work_path + "\\BF_error_class.xlsx"), sheet_name="ErrorList", usecols=["errorID", "hexLocation"])

In [ ]:
conn = pg.connect(dbname= "mydb", user= "sirius", password= "biicf", host= "localhost")
cur = conn.cursor()

In [ ]:
log_nums = len(df_logs)
for id, row in df_logs.iterrows():
    print(f"{log_nums}: {row["filename"]}")
    log_nums -= 1
    df = pd.read_csv((work_path + "\\InstrumentLog\\" + row["filename"]), sep="\t", usecols=selected_columns, dtype=col_type)
    if df.empty:
        continue
    df = df.loc[df.Type.isin(["87", "69", "73"])]
    df["DateTime"] = pd.to_datetime(df.DateTime + 28800, unit="s")
    start_time = df.iloc[0,1]
    first_line = df.iloc[[0]]
    last_line = df.iloc[[-1]]
    df = df.loc[df.Type.isin(["87", "69"])]
    df = pd.merge(df, dferror, how="left", left_on="ResourceId", right_on="errorID")
    df.dropna(subset={"errorID"}, axis=0, inplace=True)
    df["Parameters"] = df.Parameters.fillna("")
    df["Parameters"] = df.Parameters.map(clean_param)
    df["Parameters"] = df.Parameters.map(param_to_list)
    df = df.apply(to_hex, axis=1)
    df["Parameters"] = df.Parameters.map(strlist)
    df.drop(["errorID", "hexLocation"], axis=1, inplace=True)
    df = pd.concat([first_line, df, last_line])
    df["Type"] = df.Type.map(code_type)
    df["sn"] = row["sn"]
    df.drop_duplicates(subset=["DateTime", "ResourceId", "sn"], inplace=True, keep="first")

    cur.execute("SELECT MAX(datetime) FROM service.biofls_applog WHERE bf_sn = %s", (row["sn"],))
    last_time_db = cur.fetchone()[0]

    if not bool(last_time_db):
        for idx, line in df.iterrows():
            cur.execute("INSERT INTO service.biofls_applog \
                        (bf_sn, type, datetime, error_code, param) VALUES (%s, %s, %s, %s, %s)", 
                        (line["sn"], line["Type"], line["DateTime"], line["ResourceId"], line["Parameters"])
            )
    elif start_time > last_time_db + pd.Timedelta(10, "d"):
        cur.execute("DELETE FROM service.biofls_applog WHERE bf_sn = %s", (row["sn"],))
        for idx, line in df.iterrows():
            cur.execute("INSERT INTO service.biofls_applog \
                        (bf_sn, type, datetime, error_code, param) VALUES (%s, %s, %s, %s, %s)", 
                        (line["sn"], line["Type"], line["DateTime"], line["ResourceId"], line["Parameters"])
            )
    else:
        df = df[df["DateTime"] > last_time_db]
        for idx, line in df.iterrows():
            cur.execute("INSERT INTO service.biofls_applog \
                        (bf_sn, type, datetime, error_code, param) VALUES (%s, %s, %s, %s, %s)", 
                        (line["sn"], line["Type"], line["DateTime"], line["ResourceId"], line["Parameters"])
            )

In [ ]:
conn.commit()

In [ ]:
cur.close()
conn.close()